In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#reviwing the benchmark it seem dataset is switched
# https://research.unsw.edu.au/projects/unsw-nb15-dataset

train_df = pd.read_csv("../input/unsw-nb15/UNSW_NB15_testing-set.csv")
test_df = pd.read_csv("../input/unsw-nb15/UNSW_NB15_training-set.csv")

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df = train_df.drop(["id"], axis=1)
test_df = test_df.drop(["id"], axis=1)

In [ ]:
def data_preprocessing_attack_cat(data):
    data["attack_cat"] = data["attack_cat"].fillna('Normal')
    data["attack_cat"] = data["attack_cat"].replace(' Fuzzers','Fuzzers')
    data["attack_cat"] = data["attack_cat"].replace(' Fuzzers ','Fuzzers')
    data["attack_cat"] = data["attack_cat"].replace(' Reconnaissance','Reconnaissance')
    data["attack_cat"] = data["attack_cat"].replace(' Reconnaissance ','Reconnaissance')
    data["attack_cat"] = data["attack_cat"].replace(' Shellcode','Shellcode')
    data["attack_cat"] = data["attack_cat"].replace(' Shellcode ','Shellcode')
    data["attack_cat"] = data["attack_cat"].replace('Backdoors','Backdoor')

    return data

In [ ]:
train_df = data_preprocessing_attack_cat(train_df)
test_df = data_preprocessing_attack_cat(test_df)

In [ ]:
numerical_cols = train_df.select_dtypes(exclude=["object"]).columns
numerical_cols

In [ ]:
train_df[numerical_cols] = train_df[numerical_cols].fillna(0)
test_df[numerical_cols] = test_df[numerical_cols].fillna(0)

In [ ]:
categorical_cols = train_df.select_dtypes(include=["object"]).columns
categorical_cols = list(categorical_cols)
categorical_cols.remove("attack_cat")
categorical_cols

In [ ]:
train_df[categorical_cols] = train_df[categorical_cols].replace('-', "None")
test_df[categorical_cols] = test_df[categorical_cols].replace('-', "Noonne")

In [ ]:
train_df[categorical_cols] = train_df[categorical_cols].fillna("None")
test_df[categorical_cols] = test_df[categorical_cols].fillna("None")

In [ ]:
from sklearn.preprocessing import OneHotEncoder

def one_hot_encode_fit_col(data, col):
    values = data[col].unique()
    for index, v_col in enumerate(values):
        values[index] = col+"_"+v_col
        data[col+"_"+v_col] = np.zeros(len(data))
    enc = OneHotEncoder(handle_unknown='ignore')
    data[values] = enc.fit_transform(data[col].values.reshape(-1,1)).toarray()
    data = data.drop([col],axis=1)
    return data, enc, values

In [ ]:
def one_hot_encode_transform_col(data, col, enc, col_values):
    for v_col in col_values:
        data[v_col] = np.zeros(len(data))
    data[col_values] = enc.transform(data[col].values.reshape(-1,1)).toarray()
    data = data.drop([col],axis=1)
    return data

In [ ]:
def label_encoder_fit_col(data, col):
    values = ["out"]
    values.extend(data[col].unique())
    dict_values = {k: v for v, k in enumerate(values)}
    print(dict_values)
    data[col] = data[col].apply(lambda x: dict_values[x])
    data[col] = data[col].astype("int32")
    return data, dict_values

In [ ]:
def label_encoder_transform_col(data, col, dict_values):
    data[col] = data[col].apply(lambda x: dict_values[x] if x in dict_values else 0)
    data[col] = data[col].astype("int32")
    return data

In [ ]:
# train_df, proto_enc, col_values = one_hot_encode_fit_col(train_df, "proto")
# test_df = one_hot_encode_transform_col(test_df, "proto", proto_enc, col_values)

train_df, dict_values = label_encoder_fit_col(train_df, "proto")
test_df = label_encoder_transform_col(test_df, "proto", dict_values)

In [ ]:
# train_df, service_enc, col_values = one_hot_encode_fit_col(train_df, "service")
# test_df = one_hot_encode_transform_col(test_df, "service", service_enc, col_values)

train_df, dict_values = label_encoder_fit_col(train_df, "service")
test_df = label_encoder_transform_col(test_df, "service", dict_values)

In [ ]:
# train_df, state_enc, col_values = one_hot_encode_fit_col(train_df, "state")
# test_df = one_hot_encode_transform_col(test_df, "state", state_enc, col_values)

train_df, dict_values = label_encoder_fit_col(train_df, "state")
test_df = label_encoder_transform_col(test_df, "state", dict_values)

In [ ]:
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [ ]:
X_train = train_df.drop(["label", "attack_cat"], axis=1)
y_train = train_df["attack_cat"]
#y_train = train_df["label"]

X_test = test_df.drop(["label", "attack_cat"], axis=1)
y_test = test_df["attack_cat"]
#y_test = test_df["label"]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
np.unique(y_train)

In [ ]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                                 classes=np.unique(y_train),
                                                 y=y_train)

class_weights = {k: v for k,v in enumerate(class_weights)}
class_weights

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical 

cls_label_encoder = LabelEncoder()
y_train = cls_label_encoder.fit_transform(y_train)
y_test = cls_label_encoder.transform(y_test)

In [ ]:
cls_label_encoder.classes_

In [ ]:
input_shape = X_train.shape[1:]
n_classes = len(np.unique(y_train))

In [ ]:
y_train = to_categorical(y_train, num_classes=n_classes)
y_test = to_categorical(y_test, num_classes=n_classes)

In [ ]:
y_train.shape

In [ ]:
!pip install livelossplot

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support,confusion_matrix, classification_report

def print_score(y_pred, y_real):
    print("Accuracy: ", accuracy_score(y_real, y_pred))

    print()
    print("Macro precision_recall_fscore_support (macro) average")
    print(precision_recall_fscore_support(y_real, y_pred, average="macro"))

    print()
    print("Macro precision_recall_fscore_support (micro) average")
    print(precision_recall_fscore_support(y_real, y_pred, average="micro"))

    print()
    print("Macro precision_recall_fscore_support (weighted) average")
    print(precision_recall_fscore_support(y_real, y_pred, average="weighted"))
    
    print()
    print("Confusion Matrix")
    print(confusion_matrix(y_real, y_pred))

    print()
    print("Classification Report")
    print(classification_report(y_real, y_pred, target_names=cls_label_encoder.classes_))

In [ ]:
from keras.layers import Conv1D, MaxPool1D, Concatenate, BatchNormalization, Activation, Input, Add, \
                         GlobalAveragePooling1D, Dense
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from livelossplot import PlotLossesKeras
from tensorflow.keras.metrics import Recall, Precision
import keras
import time

In [ ]:
import keras.backend as K

def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Dense
from tensorflow.keras.layers import AvgPool1D, GlobalAveragePooling1D, MaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate, GRU, Reshape
import tensorflow.keras.backend as K

In [ ]:
def bn_rl_conv(x,filters,kernel=1,strides=1):
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters, kernel, strides=strides,padding = 'same')(x)
    return x

In [ ]:
def dense_block(x, repetition, filters):        
    for _ in range(repetition):
        y = bn_rl_conv(x, 4*filters)
        y = bn_rl_conv(y, filters, 3)
        x = concatenate([y,x])
    return x

In [ ]:
def transition_layer(x):
    x = bn_rl_conv(x, K.int_shape(x)[-1] //2 )
    x = AvgPool1D(2, strides = 2, padding = 'same')(x)
    return x

In [ ]:
#Densenet121    
def build_densenet(input_shape, n_classes, filters = 32):
    input = Input (input_shape)
    x = Conv1D(64, 7, strides = 2, padding = 'same')(input)
    x = MaxPool1D(3, strides = 2, padding = 'same')(x)   
    for repetition in [6,12,24,16]:
        
        d = dense_block(x, repetition, filters)
        x = transition_layer(d)    

    x = GlobalAveragePooling1D()(d)
    output = Dense(n_classes, activation = 'softmax')(x)   
    model = Model(input, output)
    return model

In [ ]:
from tensorflow.keras.metrics import Recall, Precision
import tensorflow.keras as keras

filters = 32
clf = build_densenet(input_shape, n_classes, filters = 32)
clf.compile(optimizer=keras.optimizers.Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall(), f1_score])   

clf.summary()

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(clf, to_file="model_fig.jpg", show_shapes=True)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from livelossplot import PlotLossesKeras


model_weights_file_path = "best_model_weights.h5"
checkpoint = ModelCheckpoint(filepath=model_weights_file_path, monitor="val_accuracy", verbose=1, save_best_only=True, mode="max", save_weights_only=True)
early_stopping = EarlyStopping(monitor="val_accuracy", mode="max", verbose=1, patience=20)
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, mode='max', min_delta=0.0001, cooldown=0, min_lr=0)
plotlosses = PlotLossesKeras()

call_backs = [checkpoint, early_stopping, lr_reduce, plotlosses]


In [ ]:
EPOCHS = 50
BATCH_SIZE = 128

history = clf.fit(X_train, y_train, 
                    validation_data=(X_test, y_test),
                    #validation_split=0.1,
                    epochs=EPOCHS, 
                    batch_size=BATCH_SIZE,
                    callbacks=call_backs,
                    class_weight=class_weights,
                    verbose=1)

In [ ]:
y_hat = = clf.predict(X_test)

In [ ]:
y_hat = np.argmax(y_hat, axis=1)
y_true = np.argmax(y_test, axis=1)

print_score(y_hat, y_true)